##### 1. Setup Chronos-ZS Benchmark

In addition to tirex you need to install the dependicies in [./requirements.txt](requirements.txt)

`pip install -r ./examples/chronos_zs/requirements.txt`

##### 2. Load Model and Run Benchmark

In [ ]:
import os
#os.environ["TIREX_NO_CUDA"] = "1"  # Turns off slstm cuda kernels if you have problems but be aware of the downsides! (see Repository FAQ)
from tirex import load_model, ForecastModel
model : ForecastModel = load_model("NX-AI/TiRex", device="cuda:0")

In [ ]:
import fev
import time
import datasets
import pandas as pd

def eval_task(model, task):
    past_data, _ = task.get_input_data(trust_remote_code=True)
    quantile_levels = task.quantile_levels
    past_data = past_data\
        .with_format("torch")\
        .cast_column(task.target_column, datasets.Sequence(datasets.Value("float32")))[task.target_column]
    loaded_data = [t for t in past_data]

    start_time = time.monotonic()
    quantiles, means = model.forecast(loaded_data, quantile_levels=quantile_levels, prediction_length=task.horizon)
    inference_time = time.monotonic() - start_time
    predictions_dict = {"predictions": means}
    for idx, level in enumerate(quantile_levels):
        predictions_dict[str(level)] = quantiles[:, :, idx]  # [num_items, horizon]

    predictions = datasets.Dataset.from_dict(predictions_dict)
    return predictions, inference_time

benchmark = fev.Benchmark.from_yaml(
    "https://raw.githubusercontent.com/autogluon/fev/refs/heads/main/benchmarks/chronos_zeroshot/tasks.yaml"
)
summaries = []
for task in benchmark.tasks:
    predictions, inference_time = eval_task(model, task)
    evaluation_summary = task.evaluation_summary(
        predictions,
        model_name="TiRex",
        inference_time_s=inference_time,
    )
    print(evaluation_summary)
    summaries.append(evaluation_summary)


summaries = pd.DataFrame(summaries)